# Welcome

In this tutorial, we will estimate a few simple choice models to get a better idea of travel behaviour and people's willingness to pay and trade off various different attributes.

Start by importing the appropriate libraries. For this exercise, you should import:
- pandas
- numpy
- biogeme

In [ ]:
import pandas as pd
import numpy as np
import biogeme.models as models
import biogeme.database as db
from biogeme.expressions import Beta, log, exp
import biogeme.biogeme as bio

Import the dataset from the game we played before lunch.

In [ ]:
df = pd.read_csv('data_stacked.csv')
df.fillna(0, inplace=True)

In [ ]:
print(df.head())

## Choice model estimation

Give your model a name. This will allow you to trace it back later to review previous results. If you do not change the name, biogeme will attach a serial number after the name, to prevent overwriting of existing model outputs.

In [ ]:
name = 'my_model_1'

Define the database and all the columns as variables that can be used with their respective column names.

In [ ]:
database = db.Database('TMC',df)
globals().update(database.variables)

Define the parameters to be estimated. The inputs of the Beta function are:
- name (usually the same as used in the equation, for clarity)
- starting value (keep at 0)
- lower bound (None, unless otherwise decided)
- upper bound (None, unless otherwise decided)
- estimate (0) or not (1) the specific parameter

In [ ]:
ASC_CAR = Beta('ASC_CAR',0,None,None,0)
ASC_PT = Beta('ASC_PT',0,None,None,0)

B_TIME = Beta('B_TIME',0,None,None,0)
B_COST = Beta('B_COST',0,None,None,0)

Define the utility functions of the three alternatives. In this case, V1 represents bike, V2 public transport and V3 the car.

In [ ]:
V1 =           B_TIME * bike_time + B_COST * bike_cost
V2 = ASC_PT  + B_TIME * pt_time   + B_COST * pt_cost
V3 = ASC_CAR + B_TIME * car_time  + B_COST * car_cost

Finally, we create a dictionary of all utility functions, availability of alternatives (all 1 in this case) and estimate the model.

## Coding explanation
modality:
    1=bike
    2=pt
    3=car

block_1 = trip purpose
    0 = work
    1 = leisure

block_2 = weather
    0 = dry and 15degrees
    1 = rain and 10degrees

mode_frequency:
mode_frequency_1 = car
mode_frequency_2 = bike
mode_frequency_3 = bus, tram, metro
mode_frequency_4 = train

1 = 4 or more days per week
2= 1-3 days per week
3 = 1-3 days per month
4 = 6-11 days per year
5 = 1-5 days per year
6 = (almost) never
    
car_license:
    1 = Yes
    2 = No

gender:
    1 = male
    2 = female
    3 = non-binary
    4 = rather not say


In [ ]:
V = {1: V1, 2: V2, 3: V3}
av = {1: 1,  2: 1,  3: 1}

logprob = models.loglogit(V,av,modality)

biogeme = bio.BIOGEME(database,logprob)
biogeme.modelName = name
biogeme.saveIterations = False
biogeme.generate_pickle = False
biogeme.generate_html = True
results = biogeme.estimate()
parameters = results.getEstimatedParameters()
statistics = pd.DataFrame(data=results.getGeneralStatistics()).transpose()[0]

Inspect the model statistics.

In [ ]:
print(statistics)

Inspect the parameter outcomes

In [ ]:
print(parameters)

# Value of time trade-off

To analyse the trade-offs of different parameters, we can divide them to get an idea of what they mean.

In [ ]:
VoT = parameters.loc['B_TIME','Value']/parameters.loc['B_COST','Value']
print(VoT)

This is the number of credits people are willing to pay to save 1 min of travel time.